In [32]:
#ignore file warnings
import warnings
warnings.filterwarnings("ignore") 

import numpy as np #for linear algebra
import tflearn as tfl #tensorflow fast to use
import tensorflow as tf #DL model training
import random #to add randomness
import json #dealing with json files
from gtts import gTTS #Google Text-to-Speech
import playsound #to play audio
import os #operating system dealing
import speech_recognition as sr #speech recognition pretraind model
import webbrowser as wb
from time import sleep

#natural language preprocessing kit
import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()

In [33]:
# move text to API to play as audio
def speak(text):
    tts = gTTS(text=text,lang="en")
    filename = "voice.mp3"
    tts.save(filename)
    playsound.playsound(filename)
    os.remove(filename)

In [34]:
#get audio as input to text as output
def get_audio():
    r = sr.Recognizer()
    with sr.Microphone() as sor:
        audio = r.listen(sor)
        what_u_said = ""
        
        try:
            what_u_said = r.recognize_google(audio)
        except:
            speak("speak clearly")
            
    return what_u_said

In [35]:
#reading json file
with open("intents.json") as file:
    data = json.load(file)

In [36]:
#text preprocessing
words = []
labels = []
docs_x = []
docs_y = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent["tag"])
        
    if intent['tag'] not in labels:
        labels.append(intent['tag'])


In [37]:
words = [stemmer.stem(w.lower()) for w in words if w != "?"]
words = sorted(list(set(words)))

labels = sorted(labels)

In [38]:
#traing data 

training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []

    wrds = [stemmer.stem(w.lower()) for w in doc]

    for w in words:
        if w in wrds:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)

In [39]:
#changing the data to a numpy array
training = np.array(training)
output = np.array(output)

In [40]:
# modelhyperparameters
tf.reset_default_graph()

net = tfl.input_data(shape=[None, len(training[0])])
net = tfl.fully_connected(net, 8)
net = tfl.fully_connected(net, 8)
net = tfl.fully_connected(net, len(output[0]), activation="softmax")
net = tfl.regression(net)

model = tfl.DNN(net)

In [41]:
model.fit(training, output, n_epoch=1000, batch_size=8, show_metric=True)
#you can increase the n_epoch to get better accuracy
model.save("model.tflearn")

Training Step: 4999  | total loss: 1.68860 | time: 0.019s
| Adam | epoch: 1000 | loss: 1.68860 - acc: 0.8876 -- iter: 32/38
Training Step: 5000  | total loss: 1.52487 | time: 0.022s
| Adam | epoch: 1000 | loss: 1.52487 - acc: 0.8989 -- iter: 38/38
--
INFO:tensorflow:C:\Users\zd\Desktop\pro\Remarkable-projects--master\Google assistant backend clone\model.tflearn is not in all_model_checkpoint_paths. Manually adding it.


In [42]:
#model.load("model.tflearn")

In [43]:
#bag of words
def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]

    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]

    for se in s_words:
        for i, w in enumerate(words):
            if w == se:
                bag[i] = 1
            
    return np.array(bag)

In [44]:
# i wanna know ur name

try:
    speak("hello {}".format(name.split()[-1]))
    
except:
    speak("what is your first name")
    name = get_audio()
    speak("hello {}".format(name.split()[-1]))
    
print("Start talking with the bot!")

while True:
    
    print("_____________________________ \nyour turn : ")
    
    inp = get_audio()
        
    results = model.predict([bag_of_words(inp, words)])
    results_index = np.argmax(results)
    tag = labels[results_index]

    print("you : ", inp)
    
    for tg in data["intents"]:
        if tg['tag'] == tag:
            responses = tg['responses']
    l=random.choice(responses)
        
    if l == "Goodbye":
        speak(l)
        print("jessica : ",l)
        break
    
    if l == "YouTube":
        speak("what is your search please")
        print("jessica : what is your search please")

        get_1 = get_audio()
        wb.open("https://www.youtube.com/results?search_query={}".format(get_1))
        break
        
    if l =="Google":
        speak("what is your search please")
        print("jessica : what is your search please")
        
        get_1 = get_audio()
        wb.open("https://www.google.com/search?q={}".format(get_1))
        break
        
    if l =="":
        speak("speak clearly")
        print("jessica : speak clearly")
            
    else:
        speak(l)
        print("jessica : ",l)

Start talking with the bot!
_____________________________ 
your turn : 
you :  hello how are you
jessica :  after calculations i am good
_____________________________ 
your turn : 
you :  what is your name
jessica :  You can call me jessica
_____________________________ 
your turn : 
you :  how old are you Jessica
jessica :  it is my first year
_____________________________ 
your turn : 
you :  tell me a joke
jessica :  Why couldn't the bicycle stand up by itself?....It was two tired.
_____________________________ 
your turn : 
you :  I want to search on a video
jessica : what is your search please
